# 🧪 Tutorial: Simulating Structural Drug Resistance Using Public Cancer Data (Updated)

## 🎯 Goal
Use real-world public mutation datasets (e.g., MSK-CHORD, TCGA) to:
- Model mutant protein structures (e.g., EGFR)
- Dock FDA-approved drugs
- Score drug-target binding affinities
- Identify early resistance flags


## 🗂️ Step 1A: Download and Parse Mutation Data for P-0076655

The file you downloaded from cBioPortal (MSK-CHORD, 2024) is a TSV file with columns such as `Gene`, `Protein Change`, `Sample ID`.

You can extract EGFR mutations like this:




In [9]:
import json

import pandas as pd

# Load the downloaded mutation file
df = pd.read_csv("data/input/P0076655.tsv", sep="\t")  # update path if needed

# Filter for relevant EGFR mutations from patient P-0076655
mutations = (
    df[
        (df["Sample ID"].str.contains("P-0076655"))
        & (df["Gene"] == "EGFR")
        & (df["Protein Change"].notna())
    ]["Protein Change"]
    .unique()
    .tolist()
)

# Save to JSON
mutation_data = {"gene": "EGFR", "mutations": mutations}
with open("work/mutations.json", "w") as f:
    json.dump(mutation_data, f, indent=2)

print(mutation_data)

{'gene': 'EGFR', 'mutations': ['L858R', 'T790M']}



This will create:
```json
{
  "gene": "EGFR",
  "mutations": ["L858R", "T790M"]
}
```

In [ ]:
# Or Call the script directly
!python scripts/parse_mutations.py data/input/P0076655.tsv > work/mutations.json

## Download the EGFR FASTA

In [6]:
import os

# Download the EGFR FASTA file from UniProt using wget
fasta_url = "https://www.uniprot.org/uniprot/P00533.fasta"
output_path = "data/input/egfr_wildtype.fasta"

# Use wget to download the FASTA file
exit_code = os.system(f"wget -O {output_path} {fasta_url}")

# Check if download was successful
result_msg = (
    "✅ EGFR wild-type FASTA downloaded successfully." if exit_code == 0 else "❌ Download failed."
)


print(result_msg)

--2025-05-04 14:57:23--  https://www.uniprot.org/uniprot/P00533.fasta
Resolving www.uniprot.org (www.uniprot.org)... 193.62.193.81
Connecting to www.uniprot.org (www.uniprot.org)|193.62.193.81|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://rest.uniprot.org/uniprot/P00533.fasta [following]
--2025-05-04 14:57:23--  https://rest.uniprot.org/uniprot/P00533.fasta
Resolving rest.uniprot.org (rest.uniprot.org)... 193.62.193.81
Connecting to rest.uniprot.org (rest.uniprot.org)|193.62.193.81|:443... connected.
HTTP request sent, awaiting response... 

✅ EGFR wild-type FASTA downloaded successfully.


301 Moved Permanently
Location: https://rest.uniprot.org/uniprotkb/P00533.fasta [following]
--2025-05-04 14:57:24--  https://rest.uniprot.org/uniprotkb/P00533.fasta
Reusing existing connection to rest.uniprot.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1328 (1.3K) [text/plain]
Saving to: ‘data/input/egfr_wildtype.fasta’

     0K .                                                     100%  106M=0s

2025-05-04 14:57:24 (106 MB/s) - ‘data/input/egfr_wildtype.fasta’ saved [1328/1328]



## Download Osimertinib Structure

In [10]:
import os

# Ensure the directory exists
os.makedirs("data/input", exist_ok=True)

# Download Osimertinib SDF file from PubChem
sdf_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/CID/71496458/SDF"
output_path = "data/input/osimertinib.sdf"

# Use wget to download the SDF file
exit_code = os.system(f"wget -O {output_path} {sdf_url}")

# Check if download was successful
result_msg = (
    "✅ Osimertinib SDF downloaded successfully." if exit_code == 0 else "❌ Download failed."
)

print(result_msg)

--2025-05-04 19:13:50--  https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/CID/71496458/SDF
Resolving pubchem.ncbi.nlm.nih.gov (pubchem.ncbi.nlm.nih.gov)... 130.14.29.110
Connecting to pubchem.ncbi.nlm.nih.gov (pubchem.ncbi.nlm.nih.gov)|130.14.29.110|:443... connected.
HTTP request sent, awaiting response... 

✅ Osimertinib SDF downloaded successfully.


200 OK
Length: unspecified [chemical/x-mdl-sdfile]
Saving to: ‘data/input/osimertinib.sdf’

     0K .........                                              79.2M=0s

2025-05-04 19:13:50 (79.2 MB/s) - ‘data/input/osimertinib.sdf’ saved [9388]



In [ ]:
# Convert SDF to PDBQT using Open Babel if used by RosettDock
!obabel osimertinib.sdf -O osimertinib.mol2

## 🧬 Step 2: Prepare the Mutated FASTA Sequence

In [13]:
import json
import os

from Bio import SeqIO

# Paths
wildtype_fasta = "data/input/egfr_wildtype.fasta"
mutations_json_path = "work/mutations.json"
mutated_fasta_path = "work/egfr_mutant.fasta"

# Load JSON
with open(mutations_json_path) as f:
    mutation_data = json.load(f)

record = SeqIO.read(wildtype_fasta, "fasta")
seq = list(str(record.seq))

mutation_map = {"L858R": (857, "R"), "T790M": (789, "M")}

for mutation in mutation_data["mutations"]:
    if mutation in mutation_map:
        idx, new_aa = mutation_map[mutation]
        seq[idx] = new_aa

record.seq = "".join(seq)
record.id = "EGFR_" + "_".join(mutation_data["mutations"])
record.description = "EGFR with " + ", ".join(mutation_data["mutations"])
SeqIO.write(record, mutated_fasta_path, "fasta")


print("✅ Mutated FASTA created:", record.id)

✅ Mutated FASTA created: EGFR_L858R_T790M


/Users/ahb/Documents/Incorp Algentics/projects/SRS_Proposal/.venv/lib/python3.10/site-packages/Bio/SeqRecord.py:354: BiopythonDeprecationWarning: Using a string as the sequence is deprecated and will raise a TypeError in future. It has been converted to a Seq object.
  warnings.warn(


# Step 3: Generate a Structure for the Mutated Protein

In [17]:
# Read mutated FASTA sequence from file
from Bio import SeqIO
from scripts.submit_alphafold3_job import submit_alphafold3_job

record = SeqIO.read("work/egfr_mutant.fasta", "fasta")
mutated_seq = str(record.seq)

print("Mutated sequence:", mutated_seq)
print(record.id)

Mutated sequence: MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGSCQKCDPSCPNGSCWGAGEENCQKLTKIICAQQCSGRCRGKSPSDCCHNQCAAGCTGPRESDCLVCRKFRDEATCKDTCPPLMLYNPTTYQMDVNPEGKYSFGATCVKKCPRNYVVTDHGSCVRACGADSYEMEEDGVRKCKKCEGPCRKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCVSCRNVSRGRECVDKCNLLEGEPREFVENSECIQCHPECLPQAMNITCTGRGPDNCIQCAHYIDGPHCVKTCPAGVMGENNTLVWKYADAGHVCHLCHPNCTYGCTGPGLEGCPTNGPKIPSIATGMVGALLLLLVVALGIGLFMRRRHIVRKRTLRRLLQERELVEPLTPSGEAPNQALLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLIMQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGRAKLLGAEEKEYHAEGGKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMVKCWMIDADSRPKFRELIIEFSKMARDPQRYLVIQ

In [18]:
# Submit AlphaFold3 job using sequence from file
job_info = submit_alphafold3_job(
    sequence_aa={"port1": mutated_seq}
    # ,ligand_file="data/input/osimertinib.sdf"  # Optional: can be None
    # ,job_note="AlphaFold3 prediction for EGFR mutant with Osimertinib"
)

✅ AlphaFold3 job submitted successfully.


AttributeError: 'str' object has no attribute 'get'

In [ ]:
# python scripts/run_alphafold.py \
#   --fasta_paths=work/egfr_L858R_T790M.fasta \
#   --output_dir=work/ \
#   --model_preset=monomer

# Step 4: Generate the Docking Score Between the Drug and the Mutated Protein Structure

In [ ]:
from scripts.submit_gnina_job import submit_gnina_job

# Option 1: AlphaFold3 already includes ligand
submit_gnina_job(
    receptor_file="work/af3_complex.pdb",
    job_note="GNINA: rescore AlphaFold3-predicted EGFR-ligand complex",
)

In [ ]:
# Option 2: Use GNINA to dock ligand separately
submit_gnina_job(
    receptor_file="work/mutant_structure.pdb",
    ligand_file="data/input/osimertinib.sdf",
    job_note="GNINA: dock Osimertinib into EGFR mutant",
)

In [ ]:
# !python scripts/rosetta_docking.py \
#   --structure work/mutant_structure.pdb \
#   --mutations work/mutations.json \
#   --ligand input/osimertinib.mol2 \
#   --out work/docking_scores.json

# Step 5: Scoring Binding using ProteinMPNN
This gives you a comprehensive structural resistance prediction

In [ ]:
from scripts.submit_proteinmpnn_job import submit_proteinmpnn_job

job_info = submit_proteinmpnn_job(
    structure_file="work/mutant_structure.pdb",
    job_note="Scoring EGFR_L858R_T790M fold stability with ProteinMPNN",
)

In [ ]:
# python scripts/run_proteinmpnn_scoring.py \
#   --structure work/mutant_structure.pdb \
#   --out work/stability_score.json

# Step 6: Report Generation

In [ ]:
# python scripts/generate_report.py \
#   --docking work/docking_scores.json \
#   --stability work/stability_score.json \
#   --out reports/final_report.pdf

## 🧾 Step 8: Validation Using Real Outcome
From the patient profile (P-0076655):
- Given Drug: Osimertinib
- Clinical Outcome: Documented PFS > 6 months

Compare your ΔΔG predictions with this real outcome to validate your model.

Also validate against:
- Literature-based ΔΔG thresholds (e.g., >3 kcal/mol = likely resistance)
- Known clinical response patterns for EGFR T790M patients

In [ ]:
snakemake -j 4 --rerun-incomplete